In [1]:
import pyarrow as pa

In [2]:
import pandas as pd
from databricks import koalas as ks
from pyspark.sql import SparkSession

In [3]:
data = {'a': [1, 2, 3, 4, 5, 6],
        'b': [100, 200, 300, 400, 500, 600],
        'c': ["one", "two", "three", "four", "five", "six"]}

index = [10, 20, 30, 40, 50, 60]

In [4]:
pdf = pd.DataFrame(data, index=index)
# from a pandas dataframe
kdf = ks.from_pandas(pdf)

In [5]:
# start from raw data
kdf = ks.DataFrame(data, index=index)